In [22]:
import numpy as np
from scipy.integrate import solve_ivp, odeint

In [23]:
def dUdt(U, t, a, b):
        x = U[:len(a)]
        y = U[len(a):]

        dxdt = a*x - x*y
        dydt = b*x*y - y

        return np.concatenate([dxdt, dydt])

In [24]:
a = np.random.uniform(-10, 10, 1000)
b = np.random.uniform(-10, 10, 1000)
x0, y0 = np.array([0.5] * 1000), np.array([0.5] * 1000)
S0 = np.concatenate([x0, y0])
tspan = np.linspace(0, 10, 100)

In [25]:
sol = odeint(dUdt, S0, tspan, args=(a, b))

In [120]:
import numpy as np 
from sklearn import metrics
from scipy.stats import energy_distance, cramervonmises_2samp
from scipy.special import kl_div
from scipy.spatial.distance import pdist, squareform, cdist

def wasserstein_distance(simulated_sample: np.ndarray, observed_sample: np.ndarray) -> float:
    # Mean Difference between simulated and observed
    distance = np.mean(np.abs(simulated_sample - observed_sample))
    return distance

def maximum_mean_discrepancy(simulated_sample: np.ndarray, observed_sample: np.ndarray, gamma = 1.0) -> float:
    n = simulated_sample.shape[1]
    distances_XX = np.zeros((n, 100, 100))  # Distances within simulated_sample
    distances_YY = np.zeros((n, 100, 100))  # Distances within observed_sample
    distances_XY = np.zeros((n, 100, 100))  # Distances between simulated_sample and observed_sample

    for i in range(n):
        distances_XX[i] = squareform(pdist(simulated_sample[:, i, np.newaxis], metric='euclidean'))
        distances_YY[i] = squareform(pdist(observed_sample[:, i, np.newaxis], metric='euclidean'))
        distances_XY[i] = cdist(simulated_sample[:, i, np.newaxis], observed_sample[:, i, np.newaxis], metric='euclidean')

    # Gaussian kernel
    KXX = np.exp(-distances_XX ** 2 / (2 * gamma ** 2))
    KYY = np.exp(-distances_YY ** 2 / (2 * gamma ** 2))
    KXY = np.exp(-distances_XY ** 2 / (2 * gamma ** 2))

    mean_KXX = np.mean(KXX, axis=(1, 2))  # Mean of kernel within simulated_sample
    mean_KYY = np.mean(KYY, axis=(1, 2))  # Mean of kernel within observed_sample
    mean_KXY = np.mean(KXY, axis=(1, 2))  # Mean of kernel between simulated_sample and observed_sample

    # MMD vectorized for each column
    mmd_values = mean_KXX + mean_KYY - 2 * mean_KXY

    return mmd_values

def cramer_von_mises(simulated_sample: np.ndarray, observed_sample: np.ndarray) -> float:
    return cramervonmises_2samp(simulated_sample, observed_sample).statistic

def energy_dist(simulated_sample: np.ndarray, observed_sample: np.ndarray) -> float:
    n = simulated_sample.shape[1]
    distances_XX = np.zeros((n, 100, 100))  # Distances within array1
    distances_YY = np.zeros((n, 100, 100))  # Distances within array2
    distances_XY = np.zeros((n, 100, 100))  # Distances between array1 and array2

    for i in range(n):
        distances_XX[i] = squareform(pdist(simulated_sample[:, i, np.newaxis], metric='euclidean'))
        distances_YY[i] = squareform(pdist(observed_sample[:, i, np.newaxis], metric='euclidean'))
        distances_XY[i] = cdist(simulated_sample[:, i, np.newaxis], observed_sample[:, i, np.newaxis], metric='euclidean')

    mean_dist_XY = np.mean(distances_XY, axis=(1, 2))  # Mean distance between columns
    mean_dist_XX = np.mean(distances_XX, axis=(1, 2))  # Mean distance within array1
    mean_dist_YY = np.mean(distances_YY, axis=(1, 2))  # Mean distance within array2

    # Calculate the energy distances for each column in a vectorized way
    energy_distances = 2 * mean_dist_XY - mean_dist_XX - mean_dist_YY

    # Output the energy distances for each column
    return energy_distances

def kullback_leibler_divergence(simulated_sample: np.ndarray, observed_sample: np.ndarray) -> float:
    # Sum of the divergence of each point
    return sum(kl_div(simulated_sample, observed_sample))

In [116]:
tempx = np.array([[0, 1, 2], [3, 4, 5], [2, 4, 6]])
tempy = np.array([[0, 0, 0], [1, 1, 1], [2, 2, 2]])

p = np.zeros((tempx.shape[1], 3, 3))

for i in range(tempx.shape[1]):
    p[i] = cdist(tempx[:,i, np.newaxis], tempy[:,i, np.newaxis])

np.mean(p, axis=(1, 2))

array([1.33333333, 2.22222222, 3.33333333])

In [27]:
x_sol = sol[:, :len(a)] # Prey
y_sol = sol[:, len(a):] # Predator

In [28]:
observed = odeint(dUdt, (0.5, 0.5), tspan, args=([1], [1]))

In [29]:
x_observed = observed[:, 0]
x_obs_mod = np.tile(x_observed, (1000, 1)).T
y_observed = observed[:, 1]
y_obs_mod = np.tile(y_observed, (1000, 1)).T

In [121]:
temp = maximum_mean_discrepancy(x_sol, x_obs_mod)

In [125]:
max(temp), min(temp)

(0.7129716922897895, 0.0001872446591306698)

In [31]:
x_sol.shape

(100, 1000)